In [1]:
from datetime import datetime, timedelta, date
import pytz
import hopsworks
import src.config as cfg
import pandas as pd

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# Define Argentina's timezone (GMT-3)
argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')

# Get the current date and time in Argentina
current_time_in_argentina = datetime.now(argentina_tz).replace(minute=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
# Round down (floor) to the nearest hour by setting minutes, seconds, and microseconds to 0
current_time_in_argentina = pd.to_datetime(current_time_in_argentina)
current_time_in_argentina

Timestamp('2024-10-02 18:00:00')

In [4]:
df = pd.read_csv('222324_together_total_pax.csv')
df['line'] = df['line'].str[-1]
df.drop(columns='Unnamed: 0', inplace=True)
df

,hour_of_entry,line,station,total_pax
0,2022-01-01 08:00:00,A,Acoyte,21.0
1,2022-01-01 09:00:00,A,Acoyte,11.0
2,2022-01-01 10:00:00,A,Acoyte,27.0
3,2022-01-01 11:00:00,A,Acoyte,24.0
4,2022-01-01 12:00:00,A,Acoyte,49.0
...,...,...,...,...
929868,2024-05-31 19:00:00,C,San Juan,199.0
929869,2024-05-31 20:00:00,C,San Juan,208.0
929870,2024-05-31 21:00:00,C,San Juan,161.0
929871,2024-05-31 22:00:00,C,San Juan,253.0


In [5]:
import pandas as pd
from datetime import timedelta

# Ensure the 'hour_of_entry' column is datetime
df['hour_of_entry'] = pd.to_datetime(df['hour_of_entry'])

# Define current time in Argentina
#current_time_in_argentina = pd.to_datetime('now')

# ---- Step 1: Handle 2024 Data with 2023 Shifted ----
# Define the period of interest for 2024 (2024-06-01 to today)
start_2024 = pd.to_datetime('2024-06-01 00:00:00')
end_2024 = current_time_in_argentina

# Filter the corresponding 2023 data for the 2024 range (one year back)
df_2023_for_2024 = df[(df['hour_of_entry'] >= start_2024 - timedelta(days=366)) &
                      (df['hour_of_entry'] <= end_2024 - timedelta(days=366))]

# Shift the 2023 data forward by one year (366 days for leap year)
df_2023_for_2024['hour_of_entry'] = df_2023_for_2024['hour_of_entry'] + timedelta(days=366)

# ---- Step 2: Handle 2023-06-01 to Today Minus 1 Year with 2022 Shifted ----
# Define the period from 2023-06-01 to today minus 1 year
start_2023 = pd.to_datetime('2023-06-01 00:00:00')
end_2023_shift_period = current_time_in_argentina - timedelta(days=365)

# Filter the corresponding 2022 data for the 2023 range (one year back)
df_2022_for_2023 = df[(df['hour_of_entry'] >= start_2023 - timedelta(days=365)) &
                      (df['hour_of_entry'] <= end_2023_shift_period - timedelta(days=365))]

# Shift the 2022 data forward by one year (365 days)
df_2022_for_2023['hour_of_entry'] = df_2022_for_2023['hour_of_entry'] + timedelta(days=365)

# ---- Step 3: Keep the Original 2024 Data Between 2024-01-01 and 2024-06-01 ----
# Define the period from 2024-01-01 to 2024-06-01
start_2024_january = pd.to_datetime('2024-01-01')
end_2024_june = pd.to_datetime('2024-06-01')

# Filter the original 2024 data for this period (from January 1st to June 1st, 2024)
df_original_2024 = df[(df['hour_of_entry'] >= start_2024_january) &
                      (df['hour_of_entry'] < end_2024_june)]

# ---- Step 4: Keep the Original 2023 Data Between Today Minus 1 Year and Start of 2024 ----
# Define the period from today minus 1 year to the start of 2024
one_year_ago = current_time_in_argentina - timedelta(days=365)

# Filter the original data for this period (from today minus 1 year to the start of 2024)
df_original_2023_for_2024_period = df[(df['hour_of_entry'] >= one_year_ago) &
                                      (df['hour_of_entry'] < start_2024_january)]

# ---- Step 5: Keep the Original 2023 Data Before 2023-06-01 ----
# Filter the original data for 2023 (before June 1, 2023)
df_original_2023 = df[(df['hour_of_entry'] >= pd.to_datetime('2023-01-01')) &
                      (df['hour_of_entry'] < pd.to_datetime('2023-06-01'))]

# ---- Step 6: Handle the Original 2022 Data for Today Minus 2 Years to End of 2022 ----
# Calculate the date two years ago from today
two_years_ago = current_time_in_argentina - timedelta(days=2*365)

# Filter the original data for the period between two years ago and the end of 2022
df_original_2022 = df[(df['hour_of_entry'] >= two_years_ago) &
                      (df['hour_of_entry'] <= pd.to_datetime('2022-12-31 23:59:59'))]

# ---- Step 7: Combine the Data and Sort ----
df_combined = pd.concat([df_original_2022, df_original_2023, df_2022_for_2023, 
                         df_2023_for_2024, df_original_2024, df_original_2023_for_2024_period]).drop_duplicates(subset=['hour_of_entry', 'line', 'station'], keep='first')

# Sort the combined DataFrame by 'hour_of_entry' in ascending order
df_combined = df_combined.sort_values(by='hour_of_entry', ascending=True).reset_index(drop=True)

# The final DataFrame should now be sorted in ascending order by 'hour_of_entry'
ts_hourly_data = df_combined


2024-10-02 18:08:20,095 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2024-10-02 18:08:20,131 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
ts_hourly_data.hour_of_entry.min(), ts_hourly_data.hour_of_entry.max()

(Timestamp('2022-10-03 18:00:00'), Timestamp('2024-10-02 18:00:00'))

In [7]:
# Define the start time (15 days before the end time)
end_time = current_time_in_argentina
start_time = end_time - timedelta(days=18)

# Filter the DataFrame between the start and end times
ts_hourly_data_2024 = ts_hourly_data[(ts_hourly_data['hour_of_entry'] >= start_time) & (ts_hourly_data['hour_of_entry'] <= end_time)]

end_time = (current_time_in_argentina - timedelta(days=365)) + timedelta(hours=4)
start_time = end_time - timedelta(days=18)

ts_hourly_data_2023 = ts_hourly_data[(ts_hourly_data['hour_of_entry'] >= start_time) & (ts_hourly_data['hour_of_entry'] <= end_time)]
ts_hourly_data_f = pd.concat([ts_hourly_data_2023, ts_hourly_data_2024], ignore_index=True)
ts_hourly_data_f

,hour_of_entry,line,station,total_pax
0,2023-09-15 22:00:00,A,Saenz Pena,155.0
1,2023-09-15 22:00:00,B,Leandro N. Alem,333.0
2,2023-09-15 22:00:00,C,Avenida de Mayo,105.0
3,2023-09-15 22:00:00,C,Independencia,237.0
4,2023-09-15 22:00:00,A,San Pedrito,182.0
...,...,...,...,...
38099,2024-10-02 18:00:00,B,Malabia,1680.0
38100,2024-10-02 18:00:00,B,Dorrego,1124.0
38101,2024-10-02 18:00:00,C,Independencia,570.0
38102,2024-10-02 18:00:00,C,Mariano Moreno,817.0


In [8]:
print(len(ts_hourly_data_2023), len(ts_hourly_data_2024))

19052 19052


In [9]:
ts_hourly_data_f.hour_of_entry.min(), ts_hourly_data.hour_of_entry.max()

(Timestamp('2023-09-15 22:00:00'), Timestamp('2024-10-02 18:00:00'))

In [10]:
# connect to the project
project = hopsworks.login(
    project=cfg.HOPSWORKS_PROJECT_NAME,
    api_key_value=cfg.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=cfg.FEATURE_GROUP_NAME,
    version=cfg.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency for each line and station",
    primary_key = ['line', 'hour_of_entry', 'station'],
    event_time='hour_of_entry',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/603286
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
feature_group.insert(ts_hourly_data_f, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/38104 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: ts_stations_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/603286/jobs/named/ts_stations_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f6e7b09190>, None)